In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

hts_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2018.csv', encoding='euc-kr')
hts_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2019.csv', encoding='euc-kr')
hts_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2020.csv', encoding='euc-kr')
hts_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2021.csv', encoding='euc-kr')
hts = pd.concat([hts_2018,hts_2019,hts_2020,hts_2021], ignore_index=True)

player_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2018.csv', encoding='euc-kr')
player_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2019.csv', encoding='euc-kr')
player_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2020.csv', encoding='euc-kr')
player_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2021.csv', encoding='euc-kr')
player = pd.concat([player_2018,player_2019,player_2020,player_2021], ignore_index=True)

hitter_base_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2018.csv', encoding='euc-kr')
hitter_base_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2019.csv', encoding='euc-kr')
hitter_base_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2020.csv', encoding='euc-kr')
hitter_base_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2021.csv', encoding='euc-kr')
hitter_base = pd.concat([hitter_base_2018,hitter_base_2019,hitter_base_2020,hitter_base_2021], ignore_index=True)

team = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv', encoding='euc-kr')
schedule_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv', encoding='euc-kr')

In [3]:
# 타자 기록이 있는 선수들과 등록 선수를 pcode와 연도를 기준으로 merge
hitter = hitter_base.merge(player, on=['PCODE','GYEAR'])

In [4]:
# 이름을 개명한 선수에 대해서 같은 이름이 쓰이도록 처리
hitter.loc[hitter['NAME']=='한동민','NAME'] = '한유섬'
hitter.loc[hitter['NAME']=='백민기','NAME'] = '백동훈'
hitter.loc[hitter['NAME']=='윤대영','NAME'] = '윤형준'
hitter.loc[hitter['NAME']=='이병휘','NAME'] = '이유찬'
hitter.loc[hitter['NAME']=='나종덕','NAME'] = '나균안'
hitter.loc[hitter['NAME']=='김동욱','NAME'] = '김동명'
hitter.loc[hitter['NAME']=='이찬건','NAME'] = '이석훈'
hitter.loc[hitter['NAME']=='김지열','NAME'] = '김사연'

hitter['BIRTH']=0
# 제공받는 hitter와 스탯티즈의 연도별 선수 기록이 달라서 동명이인을 처리 못하는 사람들에 대해 미리 생년월일을 입력
# 기록이 다른 선수 명단 : 천성호, 이재원, 이주형, 김주현
hitter.loc[hitter['PCODE']==50054,'BIRTH'] = '1997-10-30'
hitter.loc[hitter['PCODE']==68106,'BIRTH'] = '1999-07-17'
hitter.loc[hitter['PCODE']==50167,'BIRTH'] = '2001-04-02'
hitter.loc[hitter['PCODE']==66749,'BIRTH'] = '1993-12-21'

In [14]:
###############
# BeautifulSoup

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# 데이터를 모을 dataframe 생성
data = pd.DataFrame(columns=['GYEAR','PCODE','NAME','T_ID','POSITION', 'day', 'vs', 'result', 'number', 'P', 'starting', 'ab', 'score', 'hit', '2b', '3b', 'hr', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hbp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'avg', 'obp', 'slg', 'ops', 'pitch', 'avLI', 'RE24', 'WPA' ])
same_name_fail_list = []

for hitter_index in tqdm(range(len(hitter))):
  year = hitter.loc[hitter_index,'GYEAR']
  player_pcode = hitter.loc[hitter_index,'PCODE']
  player_name = hitter.loc[hitter_index,'NAME']
  player_team = hitter.loc[hitter_index,'T_ID']
  player_position = hitter.loc[hitter_index,'POSITION']

  # 선수와 연도에 맞는 플레이 로그 페이지
  wp = requests.get('http://www.statiz.co.kr/player.php?opt=3&name=' + player_name + '&re=0&da=1&year=' + str(year))
  soup = BeautifulSoup(wp.text, "html.parser")

  # 동명이인 문제로 페이지에 데이터가 없다면
  row_data = soup.find_all(class_='table table-striped table-responsive table-condensed table-bordered')
  if len(row_data)==0:
    print("\n동명이인 존재 : " + player_name)
    if hitter.loc[hitter_index,'BIRTH']!=0:
      print("생일로 찾음")
      wp = requests.get('http://www.statiz.co.kr/player.php?opt=3&name=' + player_name + '&birth=' + hitter.loc[hitter_index,'BIRTH'] + '&re=0&da=1&year=' + str(year))
      soup = BeautifulSoup(wp.text, "html.parser")
    else:  
      same_name_href_list=[]
      wp = requests.get('http://www.statiz.co.kr/player.php?name=' + player_name)
      soup = BeautifulSoup(wp.text, "html.parser")
      
      for same_name_list in soup.find_all(class_='table table-striped')[0].find_all('a'):
        same_name_href_list.append(same_name_list.attrs['href'])
      flag=0
      for same_name_page in same_name_href_list:
        wp = requests.get('http://www.statiz.co.kr/'+same_name_page+'&opt=1&re=0')
        soup = BeautifulSoup(wp.text, "html.parser")
        # 현재 찾고 있는 선수의 기록과 현재 페이지의 선수 기록이 일치하면 같은 선수라는 의미이다. 
        d = str(hitter.loc[hitter_index,'GAMENUM'].item()) + str(hitter.loc[hitter_index,'PA'].item()) + str(hitter.loc[hitter_index,'AB'].item())
        if d in soup.text:
          flag=1
          # 동명이인의 선수에 대해서 생일을 기록해둔다.
          print(player_name+" 생일 저장")
          player_birth = same_name_page[-10:]
          hitter.loc[hitter['PCODE']==player_pcode, 'BIRTH'] = player_birth
          break
      # 찾은 선수의 페이지로 접근한다.
      if flag==1:
        print("기록과 일치하는 선수 찾음")
      else:
        print("기록과 일치하는 선수 못찾음")
        same_name_fail_list.append((year,player_pcode,player_name,player_team))
        continue
      wp = requests.get('http://www.statiz.co.kr/player.php?opt=3&'+same_name_page.split('?')[1]+'&re=0&da=1&year='+str(year))
      soup = BeautifulSoup(wp.text, "html.parser")

  # 데이터 테이블 접근
  row_data = soup.find_all(class_='table table-striped table-responsive  table-condensed no-space table-bordered')[0]
  row_data_tr = row_data.find_all('tr',{'class':['oddrow_stz0','evenrow_stz0']})

  # 행렬만들기
  rawdata_day = np.zeros(len(row_data_tr), dtype=object) #날짜
  rawdata_vs = np.zeros(len(row_data_tr), dtype=object) #상대팀
  rawdata_result = np.zeros(len(row_data_tr), dtype=object) #결과
  rawdata_number = np.zeros(len(row_data_tr),dtype=object) #타순
  rawdata_P = np.zeros(len(row_data_tr), dtype=object) #포지션
  rawdata_starting = np.zeros(len(row_data_tr), dtype=object) #선발
  rawdata_ab = np.zeros(len(row_data_tr), dtype=object) #타수
  rawdata_score = np.zeros(len(row_data_tr), dtype=object) #득점
  rawdata_hit = np.zeros(len(row_data_tr), dtype=object) #안타
  rawdata_2b = np.zeros(len(row_data_tr), dtype=object) #2루타
  rawdata_3b = np.zeros(len(row_data_tr), dtype=object) #3루타
  rawdata_hr = np.zeros(len(row_data_tr), dtype=object) #홈런
  rawdata_tb = np.zeros(len(row_data_tr), dtype=object) #루타
  rawdata_rbi = np.zeros(len(row_data_tr), dtype=object) #타점
  rawdata_sb = np.zeros(len(row_data_tr), dtype=object) #도루성공
  rawdata_cs = np.zeros(len(row_data_tr), dtype=object) #도루실패
  rawdata_bb = np.zeros(len(row_data_tr), dtype=object) #볼넷
  rawdata_hbp = np.zeros(len(row_data_tr), dtype=object) #사구
  rawdata_ibb = np.zeros(len(row_data_tr), dtype=object) #고의4구
  rawdata_so = np.zeros(len(row_data_tr), dtype=object) #삼진
  rawdata_gdp = np.zeros(len(row_data_tr), dtype=object) #병살타
  rawdata_sh = np.zeros(len(row_data_tr), dtype=object) #희생타
  rawdata_sf = np.zeros(len(row_data_tr), dtype=object) #희생플라이
  rawdata_avg = np.zeros(len(row_data_tr), dtype=object) #타율
  rawdata_obp = np.zeros(len(row_data_tr), dtype=object) #출루율
  rawdata_slg = np.zeros(len(row_data_tr), dtype=object) #장타율
  rawdata_ops = np.zeros(len(row_data_tr), dtype=object) #OPS
  rawdata_pitch = np.zeros(len(row_data_tr), dtype=object) #투구
  rawdata_avLI = np.zeros(len(row_data_tr), dtype=object) 
  rawdata_RE24 = np.zeros(len(row_data_tr), dtype=object) 
  rawdata_WPA = np.zeros(len(row_data_tr), dtype=object) 


  # 행 하나씩 접근, 헤더 생략
  for tr_index in range(len(row_data_tr)):
    rawdata_td_element = [text for text in row_data_tr[tr_index].stripped_strings]
    # # P 칼럼의 값이 없을때도 있음.
    # if len(rawdata_td_element) == 14:
    #   rawdata_td_element.insert(6,"")
    
    # 데이터 입력
    rawdata_day[tr_index] = rawdata_td_element[1]
    rawdata_vs[tr_index] = rawdata_td_element[2]
    rawdata_result[tr_index] = rawdata_td_element[3]+" "+rawdata_td_element[4]
    rawdata_number[tr_index] = rawdata_td_element[5]
    rawdata_P[tr_index] = rawdata_td_element[6]
    rawdata_starting[tr_index] = rawdata_td_element[7]
    rawdata_ab[tr_index] = rawdata_td_element[8]
    rawdata_score[tr_index] = rawdata_td_element[9]
    rawdata_hit[tr_index] = rawdata_td_element[10]
    rawdata_2b[tr_index] = rawdata_td_element[11]
    rawdata_3b[tr_index] = rawdata_td_element[12]
    rawdata_hr[tr_index] = rawdata_td_element[13]
    rawdata_tb[tr_index] = rawdata_td_element[14]
    rawdata_rbi[tr_index] = rawdata_td_element[15]
    rawdata_sb[tr_index] = rawdata_td_element[16]
    rawdata_cs[tr_index] = rawdata_td_element[17]
    rawdata_bb[tr_index] = rawdata_td_element[18]
    rawdata_hbp[tr_index] = rawdata_td_element[19]
    rawdata_ibb[tr_index] = rawdata_td_element[20]
    rawdata_so[tr_index] = rawdata_td_element[21]
    rawdata_gdp[tr_index] = rawdata_td_element[22]
    rawdata_sh[tr_index] = rawdata_td_element[23]
    rawdata_sf[tr_index] = rawdata_td_element[24]
    rawdata_avg[tr_index] = rawdata_td_element[25]
    rawdata_obp[tr_index] = rawdata_td_element[26]
    rawdata_slg[tr_index] = rawdata_td_element[27]
    rawdata_ops[tr_index] = rawdata_td_element[28]
    rawdata_pitch[tr_index] = rawdata_td_element[29]  
    rawdata_avLI[tr_index] = rawdata_td_element[30]
    rawdata_RE24[tr_index] = rawdata_td_element[31]
    rawdata_WPA[tr_index] = rawdata_td_element[32]

  tmp_data = pd.DataFrame()

  # tmp_data는 각 선수마다 데이터를 모은 후 data에 concat 하기 전 임시로 저장

  tmp_data['day'] = rawdata_day
  tmp_data['vs'] = rawdata_vs
  tmp_data['result'] = rawdata_result
  tmp_data['number'] = rawdata_number
  tmp_data['P'] = rawdata_P
  tmp_data['starting'] = rawdata_starting
  tmp_data['ab'] = rawdata_ab
  tmp_data['score'] = rawdata_score
  tmp_data['hit'] = rawdata_hit
  tmp_data['2b'] = rawdata_2b
  tmp_data['3b'] = rawdata_3b
  tmp_data['hr'] = rawdata_hr
  tmp_data['tb'] = rawdata_tb
  tmp_data['rbi'] = rawdata_rbi
  tmp_data['sb'] = rawdata_sb
  tmp_data['cs'] = rawdata_cs
  tmp_data['bb'] = rawdata_bb
  tmp_data['hbp'] = rawdata_hbp
  tmp_data['ibb'] = rawdata_ibb
  tmp_data['so'] = rawdata_so
  tmp_data['gdp'] = rawdata_gdp
  tmp_data['sh'] = rawdata_sh
  tmp_data['sf'] = rawdata_sf
  tmp_data['avg'] = rawdata_avg
  tmp_data['obp'] = rawdata_obp
  tmp_data['slg'] = rawdata_slg
  tmp_data['ops'] = rawdata_ops
  tmp_data['pitch'] = rawdata_pitch
  tmp_data['avLI'] = rawdata_avLI
  tmp_data['RE24'] = rawdata_RE24
  tmp_data['WPA'] = rawdata_WPA

  tmp_data['GYEAR'] = year
  tmp_data['PCODE'] = player_pcode
  tmp_data['NAME'] = player_name
  tmp_data['T_ID'] = player_team
  tmp_data['POSITION'] = player_position

  # 데이터 쌓기
  data = pd.concat([data,tmp_data], ignore_index=True)

  0%|          | 0/1098 [00:00<?, ?it/s]


동명이인 존재 : 백창수


  0%|          | 0/1098 [00:02<?, ?it/s]


IndexError: ignored

In [ ]:
data

,GYEAR,PCODE,NAME,T_ID,POSITION,day,vs,inn,pitcher,batter,P,result,before,after,LEV,REa,WPs,WPe,WPa
0,2018,60100,백창수,HH,외,2018-03-27,NC,8초,이민호,백창수,1-0,유격수 땅볼,무사 6:9,1사 6:9,0.88,-0.258,93.1,95.3,-0.022
1,2018,60100,백창수,HH,외,2018-03-28,NC,2초,구창모,백창수,2-3,볼넷,2사 2루 0:2,"2사 1,2루 0:2",1.10,0.122,71.9,70.7,0.012
2,2018,60100,백창수,HH,외,2018-03-28,NC,4초,구창모,백창수,0-0,3루수 땅볼,2사 1:2,이닝종료 1:2,0.51,-0.117,67.4,68.8,-0.013
3,2018,60100,백창수,HH,외,2018-03-28,NC,7초,유원상,백창수,2-3,유격수 땅볼 실책,[실책포함],무사 2:2,무사 1루 2:2,1.54,0.398,50.0,44.1
4,2018,60100,백창수,HH,외,2018-03-30,SK,2말,김태훈,백창수,0-1,백창수/10473 : 2루수 직선타 아웃,"1사 2,3루 1:1","2사 2,3루 1:1",1.57,-0.833,64.1,56.1,-0.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205941,2021,79608,안치홍,LT,내,2021-08-25,KIA,2초,멩덴,안치홍,1-1,2루수 땅볼,1사 0:0,2사 0:0,0.65,-0.180,52.4,54.0,-0.016
205942,2021,79608,안치홍,LT,내,2021-08-25,KIA,4초,멩덴,안치홍,1-1,유격수 땅볼,무사 1루 1:5,1사 1루 1:5,1.20,-0.380,85.1,87.8,-0.027
205943,2021,79608,안치홍,LT,내,2021-08-25,KIA,5초,멩덴,안치홍,0-0,포수 땅볼,1사 4:6,2사 4:6,0.79,-0.180,78.4,80.3,-0.019
205944,2021,79608,안치홍,LT,내,2021-08-25,KIA,7초,박진태,안치홍,1-1,유격수 내야안타,"1사 1,2루 4:7",1사 만루 4:7,2.30,0.679,84.9,77.0,0.079
